### GW灰度世界白平衡算法

灰度世界算法（Gray World)是以灰度世界假设为基础的,该假设认为对于一幅有着大量色彩变化的图像, R、 G、 B 三个分量的平均值趋于同一个灰度K。
$$Rgain = k / Ravg $$
$$Ggain = k / Gavg $$
$$Bgain = k / Bavg $$
然后通过k值来求各个通达的增益。
1. 直接给定为固定值, 取其各通道最大值的一半,即取为127或128；
2. 令 K = (Raver+Gaver+Baver)/3,其中Raver,Gaver,Baver分别表示红、 绿、 蓝三个通道的平均值。
算法的第二步是分别计算各通道的增益：
$$k =  (Bavg + Gavg + Ravg)/3$$
3. 设定G通道不变，让R和B通道往G通道上靠，即：
$$Rgain = Rave / Gave$$
$$Bgain = Bave / Gave$$
新的像素的绿色通道不变，红色和蓝色通道通过绿色通道来求

In [7]:
# _*_ coding: UTF-8 _*_
# path:

"""
作者：朱文涛
邮箱：wtzhu_13@163.com

时间：2019/05
描述：灰度世界法实现白平衡
"""
import cv2 as cv
import numpy as np

src = cv.imread('test.jpg')

# 求出各个颜色分量的平均值
b_avg = np.mean(src[:, :, 0])
g_avg = np.mean(src[:, :, 1])
r_avg = np.mean(src[:, :, 2])
# 求出灰度世界的灰度值
k = (b_avg + g_avg + r_avg)/3
# 求出各个颜色分量的增益
b_gain = k / b_avg
g_gain = k / g_avg
r_gain = k / r_avg

# 定义一个新的矩阵存放变换后的图像
src1 = np.zeros(src.shape)
src1[:, :, 0] = src[:, :, 0] * b_gain
src1[:, :, 1] = src[:, :, 1] * g_gain
src1[:, :, 2] = src[:, :, 2] * r_gain

# 计算后类型为浮点数，需要类型转换
src1 = src1.astype(np.uint8)

# 拼接两张图片，便于观察
img = np.hstack([src, src1])
cv.namedWindow('input_image', cv.WINDOW_AUTOSIZE)
cv.imshow('input_image', img)
cv.waitKey(0)
cv.destroyAllWindows()


### PR完全反射白平衡算法
图像中最亮的点为白点，那么各个通道的最大值应该趋近白色，可是设定一个K值，如255,那么图像中最两点的值应该趋于255，这样就可以计算出各通道的增益：
$$Rgain = k / Rmax $$
$$Ggain = k / Gmax $$
$$Bgain = k / Bmax $$

In [8]:
# _*_ coding: UTF-8 _*_
# path:

"""
作者：朱文涛
邮箱：wtzhu_13@163.com

时间：2019/05
描述：全反射实现白平衡
"""
import cv2 as cv
import numpy as np

src = cv.imread('test.jpg')

# 求出各个颜色分量的增益
b_gain = 255 / np.max(src[:, :, 0])
g_gain = 255 / np.max(src[:, :, 1])
r_gain = 255 / np.max(src[:, :, 2])

# 定义一个新的矩阵存放变换后的图像
src1 = np.zeros(src.shape)
src1[:, :, 0] = src[:, :, 0] * b_gain
src1[:, :, 1] = src[:, :, 1] * g_gain
src1[:, :, 2] = src[:, :, 2] * r_gain

# 计算后类型为浮点数，需要类型转换
src1 = src1.astype(np.uint8)

# 拼接两张图片，便于观察
img = np.hstack([src, src1])
cv.namedWindow('input_image', cv.WINDOW_AUTOSIZE)
cv.imshow('input_image', img)
cv.waitKey(0)
cv.destroyAllWindows()


### QCGP白平衡算法

通过GW和PR两种算法的正交组合，从而保留两者的优点，具体的算法公式如下：
$$ Kave = (Rave + Gave + Bave) / 3 $$
$$ Kmax = (Rmax + Gmax + Bmax) / 3$$
$$ u * Rave^2 + v * Rave = Kave $$
$$ u * Rmax^2 + v* Rmax = Kmax $$
这是R通道的算法公式，求出u和v然后通过以下公式换算出新的值：
$$Rnew = u * Rorg ^ 2 + v * Rorg$$

In [13]:
# _*_ coding: UTF-8 _*_
# path:

"""
作者：朱文涛
邮箱：wtzhu_13@163.com

时间：2020/04
描述：GW和PR正交组合算法白平衡QCGP
"""

import cv2 as cv
import numpy as np

src0 = cv.imread('test.jpg')
src = src0.astype(np.uint16)    # 调整一下数据类型，防止算术运算溢出

# 求出各个颜色分量的平均值
b_ave = np.mean(src[:, :, 0])
g_ave = np.mean(src[:, :, 1])
r_ave = np.mean(src[:, :, 2])

# 各个颜色分量的最大值
b_max = np.max(src[:, :, 0])
g_max = np.max(src[:, :, 1])
r_max = np.max(src[:, :, 2])

# 根据QCGP公式求出系数
k_ave = (b_ave + g_ave + r_ave)/3
k_max = (b_max + g_max + r_max)/3
k_matrix = np.mat([[k_ave], [k_max]])

# 通过矩阵求出B通道的转换矩阵，并计算出新图的B通道
b_coefficient_matrix = np.mat([[b_ave * b_ave, b_ave],
                               [b_max * b_max, b_max]])
b_conversion_matrix = b_coefficient_matrix.I * k_matrix

b = (src[:, :, 0]).transpose()
bb = (src[:, :, 0] * src[:, :, 0]).transpose()
b = np.stack((bb, b), axis=0).transpose()
b_des = np.dot(b, np.array(b_conversion_matrix))
b_des = b_des.astype(np.uint8).reshape([590, 894])

# 通过矩阵求出G通道的转换矩阵，并计算出新图的G通道
g_coefficient_matrix = np.mat([[g_ave * g_ave, g_ave],
                               [g_max * g_max, g_max]])
g_conversion_matrix = g_coefficient_matrix.I * k_matrix

g = (src[:, :, 1]).transpose()
gg = (src[:, :, 1] * src[:, :, 1]).transpose()
g = np.stack((gg, g), axis=0).transpose()
g_des = np.dot(g, np.array(g_conversion_matrix))
g_des = g_des.astype(np.uint8).reshape([590, 894])

# 通过矩阵求出R通道的转换矩阵，并计算出新图的R通道
r_coefficient_matrix = np.mat([[r_ave * r_ave, r_ave],
                               [r_max * r_max, r_max]])
r_conversion_matrix = r_coefficient_matrix.I * k_matrix

r = (src[:, :, 2]).transpose()
rr = (src[:, :, 2] * src[:, :, 2]).transpose()
r = np.stack((rr, r), axis=0).transpose()
r_des = np.dot(r, np.array(r_conversion_matrix))
r_des = r_des.astype(np.uint8).reshape([590, 894])

# 用一个新的矩阵接受新的图片，注意数据类型要和原图一致
src1 = np.zeros(src.shape).astype(np.uint8)
src1[:, :, 0] = b_des
src1[:, :, 1] = g_des
src1[:, :, 2] = r_des

# 显示图片
img = np.hstack([src0, src1])
cv.namedWindow("AWB", cv.WINDOW_AUTOSIZE)
cv.imshow("AWB", img)
cv.waitKey(0)
cv.destroyAllWindows()


### 一种自动阈值白平衡算法实现

白平衡算法通常分为两步：白色点的检测，白色点的调整。本方法采用一个动态的阀值来检测白色点。详细算法过程为：
1.  把图像w*h从RGB空间转换到YCrCb空间。
2.  选择参考白色点：
        a. 把图像分成宽高比为4:3个块(块数可选)。
        b. 对每个块，分别计算Cr，Cb的平均值Mr，Mb。
        c. 对每个块，根据Mr，Mb，用下面公式分别计算Cr，Cb的方差Dr，Db。
        d. 判定每个块的近白区域（near-white region）。
            判别表达式为：
            设一个“参考白色点”的亮度矩阵RL，大小为w*h。
             若符合判别式，则作为“参考白色点”,并把该点（i,j）的亮度（Y分量）值赋给RL(i,j)；
             若不符合，则该点的RL(i,j)值为0。
3.  选取参考“参考白色点”中最大的10%的亮度（Y分量）值，并选取其中的最小值Lu_min.
4.  调整RL，若RL(i,j)<Lu_min,  RL(i,j)=0; 否则，RL(i,j)=1；
5.  分别把R，G，B与RL相乘，得到R2，G2，B2。  分别计算R2，G2，B2的平均值，Rav，Gav，Bav；
6.  得到调整增益：  Ymax=double(max(max(Y)))/5;
                        Rgain=Ymax/Rav;
                        Ggain=Ymax/Gav;
                        Bgain=Ymax/Bav;
7.  调整原图像：Ro= R*Rgain; Go= G*Ggain; Bo= B*Bgain;

In [10]:
# _*_ coding: UTF-8 _*_
# path:

"""
作者：朱文涛
邮箱：wtzhu_13@163.com

时间：2020/04
描述：自动阈值白平衡
"""

import cv2
import numpy as np

def RGB2YCrCb(src):
    """
    RGB 转换为 YCrCb
    """
    Y = src[:, : ,0] * 0.098 + src[:, : ,1] * 0.504 + src[:, : ,2] * 0.257 +16
    Cr = - src[:, : ,0] * 0.071- src[:, : ,1] * 0.368 + src[:, : ,2] * 0.439+128
    Cb = src[:, : ,0] * 0.439 - src[:, : ,1] * 0.291 - src[:, : ,2] * 0.148 +128
    image = np.zeros(src.shape)
    image[:, :, 0] = Y
    image[:, :, 1] = Cr
    image[:, :, 2] = Cb
    return image

def computeRL(img):    
    Y = img[:, :, 0]
    Cr = img[:, :, 1]
    Cb = img[:, :, 2]

    Mr = np.mean(Cr)
    Mb = np.mean(Cb)

    Dr = np.var(Cr)
    Db = np.var(Cb)

    temp_arry = (np.abs(Cb - (Mb + Db * np.sign(Mb))) < 1.5 * Db) & (np.abs(Cr - (1.5 * Mr + Dr * np.sign(Mr))) < 1.5 * Dr)
    RL = Y * temp_arry
    return RL

def autoAWB(RL):
    b, g, r = cv2.split(src)
    # 选取候选白点数的最亮10%确定为最终白点，并选择其前10%中的最小亮度值
    L_list = list(np.reshape(RL, (RL.shape[0] * RL.shape[1],)).astype(np.int))
#     L = [i for i in L_list if i != 0]
    hist_list = np.zeros(256)
    min_val = 0
    sum = 0
    for val in L_list:
        hist_list[val] += 1

    for l_val in range(255,0,-1):
        sum += hist_list[l_val]
        if sum >= len(L_list) * 0.1:
            min_val = l_val
            break
    # 取最亮的前10%为最终的白点
    white_index = RL < min_val
    RL[white_index] = 0

    # 计算选取为白点的每个通道的增益
    b[white_index] = 0
    g[white_index] = 0
    r[white_index] = 0

    Y_max = np.max(RL)
    b_gain = Y_max / (np.sum(b) / np.sum(b>0))
    g_gain = Y_max / (np.sum(g) / np.sum(g>0))
    r_gain = Y_max / (np.sum(r) / np.sum(r>0))

    b = b * b_gain
    g = g * g_gain
    r = r * r_gain

    # 溢出处理
    b[b > 255] = 255
    g[g > 255] = 255
    r[r > 255] = 255

    res_img = cv2.merge((b,g,r))
    return res_img
    
src = cv2.imread('test.jpg')
img_data = RGB2YCrCb(src)
RL1 = computeRL(img_data)
img = autoAWB(RL1)
cv2.imwrite('1_auto.jpg', img)

# cv2.namedWindow('input_image', cv2.WINDOW_AUTOSIZE)
# src = cv2.imread('1_auto.jpg')
# cv2.imshow('input_image', src)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

True

In [4]:
def auto_whiteBalance(img):    
    b, g, r = cv2.split(img)    
    Y = src[:, : ,0] * 0.098 + src[:, : ,1] * 0.504 + src[:, : ,2] * 0.257 +16
    Cr = - src[:, : ,0] * 0.071- src[:, : ,1] * 0.368 + src[:, : ,2] * 0.439+128
    Cb = src[:, : ,0] * 0.439 - src[:, : ,1] * 0.291 - src[:, : ,2] * 0.148 +128

    Mr = np.mean(Cr)
    Mb = np.mean(Cb)

    Dr = np.var(Cr)
    Db = np.var(Cb)

    temp_arry = (np.abs(Cb - (Mb + Db * np.sign(Mb))) < 1.5 * Db) & (np.abs(Cr - (1.5 * Mr + Dr * np.sign(Mr))) < 1.5 * Dr)
    RL = Y * temp_arry


    # 选取候选白点数的最亮10%确定为最终白点，并选择其前10%中的最小亮度值
    L_list = list(np.reshape(RL, (RL.shape[0] * RL.shape[1],)).astype(np.int))
    print(L_list)
    hist_list = np.zeros(256)
    min_val = 0
    sum = 0
    for val in L_list:
        hist_list[val] += 1

    for l_val in range(255,0,-1):
        sum += hist_list[l_val]
        if sum >= len(L_list) * 0.1:
            min_val = l_val
            break
    # 取最亮的前10%为最终的白点
    white_index = RL < min_val
    RL[white_index] = 0

    # 计算选取为白点的每个通道的增益
    b[white_index] = 0
    g[white_index] = 0
    r[white_index] = 0

    Y_max = np.max(RL)
    b_gain = Y_max / (np.sum(b) / np.sum(b>0))
    g_gain = Y_max / (np.sum(g) / np.sum(g>0))
    r_gain = Y_max / (np.sum(r) / np.sum(r>0))

    b, g, r = cv2.split(img)
    b = b * b_gain
    g = g * g_gain
    r = r * r_gain

    # 溢出处理
    b[b > 255] = 255
    g[g > 255] = 255
    r[r > 255] = 255

    res_img = cv2.merge((b,g,r))
    return res_img
    
img_data = cv2.imread('test.jpg')
img = auto_whiteBalance(img_data)
cv2.imwrite('1_auto.jpg', img)


[93, 92, 90, 88, 85, 82, 81, 80, 82, 82, 82, 82, 82, 82, 88, 95, 99, 101, 103, 102, 101, 100, 100, 101, 101, 103, 105, 107, 108, 109, 111, 113, 107, 108, 109, 108, 103, 101, 106, 141, 142, 145, 139, 139, 144, 138, 133, 109, 112, 113, 115, 115, 113, 116, 116, 108, 112, 113, 109, 105, 106, 103, 99, 100, 96, 100, 101, 98, 101, 107, 111, 126, 129, 125, 116, 113, 118, 123, 125, 128, 128, 127, 128, 129, 130, 131, 133, 134, 127, 118, 107, 101, 99, 100, 100, 100, 99, 100, 100, 100, 100, 101, 100, 98, 98, 101, 103, 105, 107, 108, 108, 108, 108, 108, 107, 108, 109, 108, 107, 106, 108, 106, 107, 107, 112, 108, 106, 111, 109, 105, 101, 96, 82, 58, 39, 37, 33, 27, 23, 23, 23, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 20, 20, 20, 20, 20, 20, 20, 20, 0, 20, 20, 20, 21, 21, 21, 21, 20, 21, 22, 23, 23, 23, 24, 26, 26, 27, 27, 28, 29, 30, 28, 26, 26, 27, 27, 27, 26, 26, 26, 27, 29, 29, 30, 29, 26, 24, 25, 28, 24, 24, 26, 28, 29, 27, 27, 26, 24, 26, 32, 33, 34, 40, 49, 65, 58, 70, 94, 100, 86, 93, 110, 111, 112,

True